**Sentiment Analysis**

**I. Individual Analysis**

**Electronics**

*General Imports*

In [1]:
import nltk
from random import shuffle
from sklearn.linear_model import LogisticRegression
from nltk.metrics.scores import (precision, recall, accuracy)

**Features Construction**

Useful functions to load the different features models

In [2]:
def load_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = int(theword[1])
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [3]:
def load_binary_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list. Filters them to change it to 1 for each 
    existent element in the Bag of Words.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = 1
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [4]:
def load_lexicon(thefeatures,filename):
    '''
    Loads the tags process them to create the Features based on the
    AFINN-111 sentiment lexicon.
    '''
    lex = load_affin()
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats1 = {}
        summa = 0
        negga = 0
        pos = 0
        neg = 0
        tot = 0
        
        for word in range(len(splited)-1):
            add = False
            theword = splited[word].split(":")
            lilwords = theword[0].split("_")
            for lw in lilwords:
                if lw in lex:
                    add = True
                    if lex[lw] > 0:
                        summa += lex[lw]*int(theword[1])
                        pos += int(theword[1])
                    else:
                        negga += lex[lw]*int(theword[1])
                        neg += int(theword[1])
            tot += int(theword[1])
        feats1["point_pos"] = summa
        feats1["point_neg"] = -negga
        feats1["count_pos"] = pos
        feats1["count_neg"] = neg
        feats1["size"] = tot
        feats1["porc_pos"] = pos/tot
        feats1["porc_neg"] = neg/tot
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats1,tag))
        line = f.readline()
    f.close()

def load_affin():
    '''
    Loads the lexicon of sentiment words included in the AFINN-11 lexicon
    '''
    f = open("AFINN-111.txt", "r")
    line = f.readline()
    affin = {}
    while len(line) >0:
        splited = line.split('\t')
        affin[splited[0]] = int(splited[1])
        line = f.readline()
    return affin

*Additional Function*

Shows the most informative features for the Linear Regression Models.

Taken from the answers to: https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers Kudos to him.

In [5]:
def show_most_informative_features(vectorizer, clf, n=20):
    '''
    Gets the most informative features from a Linear Regression Model
    '''
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

**BoW**

Loads the data and creates the feature to train. Loads and uses both review files.

In [6]:
BoWFeatures = []

#Loads the positive reviews
load_bow(BoWFeatures,"reviews/electronics/positive.review")

#Loads the negative reviews 
load_bow(BoWFeatures,"reviews/electronics/negative.review")

print("Loaded",len(BoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [7]:
BoWTesting =[]

#Loads the testing reviews 
load_bow(BoWTesting,"reviews/electronics/unlabeled.review")

print("Loaded",len(BoWTesting),"reviews")

Loaded 5681 reviews


**Binary BoW**

Loads the data and creates the feature to train. Loads and uses both review files. Filters them to change it to 1 for each existent element in the Bag of Words.

In [8]:
BinaryBoWFeatures = []

#Loads the positive reviews
load_binary_bow(BinaryBoWFeatures,"reviews/electronics/positive.review")

#Loads the negative reviews 
load_binary_bow(BinaryBoWFeatures,"reviews/electronics/negative.review")

print("Loaded",len(BinaryBoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [9]:
BinaryBoWTesting =[]

#Loads the testing reviews 
load_binary_bow(BinaryBoWTesting,"reviews/electronics/unlabeled.review")

print("Loaded",len(BinaryBoWTesting),"reviews")

Loaded 5681 reviews


**Lexicon Features**

Loads the data and creates the feature to train. Loads and uses both review files. Takes into use the following Features:

- Number of positive words
- Number of negative words
- Positive score following the AFINN-111 scores
- Negative score following the AFINN-111 scores
- Size (number of words) of the review
- Percentage of positive words
- Percentage of negative words

In [10]:
LexiconFeatures = []

#Loads the positive reviews
load_lexicon(LexiconFeatures,"reviews/electronics/positive.review")

#Loads the negative reviews 
load_lexicon(LexiconFeatures,"reviews/electronics/negative.review")

print("Loaded",len(LexiconFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [11]:
LexiconTesting =[]

#Loads the testing reviews 
load_lexicon(LexiconTesting,"reviews/electronics/unlabeled.review")

print("Loaded",len(LexiconTesting),"reviews")

Loaded 5681 reviews


**Testing Zone**

Now we are gonna test the six different combinations of Features and classification algorithms

**BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Bag of Words Model.

**Training**

In [12]:
shuffle(BoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
                terrible = 1                 neg : pos    =     35.0 : 1.0
                    junk = 1                 neg : pos    =     26.3 : 1.0
                 not_buy = 1                 neg : pos    =     26.3 : 1.0
               not_worth = 1                 neg : pos    =     18.3 : 1.0
                 of_junk = 1                 neg : pos    =     17.0 : 1.0
                  refund = 1                 neg : pos    =     17.0 : 1.0
                   worst = 1                 neg : pos    =     16.6 : 1.0
            same_problem = 1                 neg : pos    =     15.7 : 1.0
                   waste = 1                 neg : pos    =     15.7 : 1.0
                 and_got = 1                 neg : pos    =     15.0 : 1.0


**Validating**

In [13]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [14]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8593557472276008
Precision: 0.8755474452554745
Recall: 0.83969198459923
F1 Score: 0.8572449526532071


**BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Bag of Words Model.

**Training**

In [15]:
shuffle(BoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [16]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [17]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8577715190987503
Precision: 0.8493010569382884
Recall: 0.871893594679734
F1 Score: 0.8604490500863559


In [18]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-0.9448	not            		1.3865	great          
	-0.9326	poor           		1.1380	excellent      
	-0.7420	bad            		0.9220	perfect        
	-0.6297	return         		0.8215	price          
	-0.6159	waste          		0.7929	best           
	-0.6118	disappointed   		0.7244	works          
	-0.6110	returned       		0.6652	good           
	-0.6073	money          		0.6492	fast           
	-0.6065	back           		0.5959	the_best       
	-0.6002	terrible       		0.5604	no_problems    


**Binary BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Binary Bag of Words Model.

**Training**

In [19]:
shuffle(BinaryBoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BinaryBoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
                terrible = 1                 neg : pos    =     36.3 : 1.0
                 not_buy = 1                 neg : pos    =     27.7 : 1.0
                    junk = 1                 neg : pos    =     20.2 : 1.0
               not_worth = 1                 neg : pos    =     19.7 : 1.0
                  refund = 1                 neg : pos    =     19.0 : 1.0
                   worst = 1                 neg : pos    =     19.0 : 1.0
                 of_junk = 1                 neg : pos    =     17.7 : 1.0
            same_problem = 1                 neg : pos    =     17.0 : 1.0
                   waste = 1                 neg : pos    =     16.8 : 1.0
                  no_way = 1                 neg : pos    =     16.3 : 1.0


**Validating**

In [20]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [21]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8568913923604999
Precision: 0.874633431085044
Recall: 0.8351417570878544
F1 Score: 0.8544315129811997


**Binary BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Binary Bag of Words Model.

**Training**

In [22]:
shuffle(BinaryBoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BinaryBoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [23]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [24]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8663967611336032
Precision: 0.8639833448993754
Recall: 0.871543577178859
F1 Score: 0.8677469942498693


In [25]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.0226	not            		1.7349	great          
	-0.9543	poor           		1.1647	excellent      
	-0.7812	bad            		0.9990	perfect        
	-0.7182	work           		0.9294	price          
	-0.6780	waste          		0.8807	best           
	-0.6526	terrible       		0.7958	works          
	-0.6406	return         		0.7172	good           
	-0.6080	disappointed   		0.7106	the_best       
	-0.5936	not_work       		0.6960	fast           
	-0.5803	returned       		0.6471	highly         


**Lexicon Features - Naive Bayes**

Trains the dataset over a Naive Bayes with a Lexicon Features Model.

**Training**

In [26]:
shuffle(LexiconFeatures)
classifier = nltk.NaiveBayesClassifier.train(LexiconFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
               count_pos = 7                 pos : neg    =     13.7 : 1.0
               count_neg = 7                 neg : pos    =     10.3 : 1.0
               count_pos = 4                 pos : neg    =     10.3 : 1.0
               point_neg = 13                neg : pos    =      7.7 : 1.0
                porc_pos = 0.04              neg : pos    =      7.0 : 1.0
               count_pos = 10                pos : neg    =      6.7 : 1.0
               point_neg = 16                neg : pos    =      6.4 : 1.0
               count_neg = 20                neg : pos    =      6.3 : 1.0
               count_pos = 22                pos : neg    =      6.3 : 1.0
               point_neg = 28                neg : pos    =      6.3 : 1.0


**Validating**

In [27]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.883
Precision: 0.8711240310077519
Recall: 0.899
F1 Score: 0.8848425196850395


**Testing**

In [28]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.725752508361204
Precision: 0.7302375044310528
Recall: 0.7210360518025901
F1 Score: 0.7256076083127861


**Lexicon Features - Linear Regression**

Trains the dataset over a Linear Regression with a Lexicon Features Model.

**Training**

In [29]:
shuffle(LexiconFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(LexiconFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [30]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.7455
Precision: 0.749238578680203
Recall: 0.738
F1 Score: 0.7435768261964735


**Testing**

In [31]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.7528604118993135
Precision: 0.7551808921671935
Recall: 0.7525376268813441
F1 Score: 0.7538569424964937


In [32]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-0.0945	count_pos      		3.9713	porc_pos       
	-0.0893	point_neg      		0.1119	porc_neg       
	0.0004	size           		0.0904	point_pos      
	0.0060	count_neg      		0.0060	count_neg      
	0.0904	point_pos      		0.0004	size           
	0.1119	porc_neg       		-0.0893	point_neg      
	3.9713	porc_pos       		-0.0945	count_pos      


**Table of Results - Electronics**

| Combination | Accuracy | Precision | Recall | F1 Score|
| --- | ----------- |----------- |----------- |----------- |
| **BOW - NB** | 0.8593557472276008 | **0.8755474452554745** | 0.83969198459923 | 0.8572449526532071 |
| **BOW - LR** | 0.8577715190987503 | 0.8493010569382884 | **0.871893594679734** | 0.8604490500863559 |
| **BBOW - NB** | 0.8568913923604999 | 0.874633431085044 | 0.8351417570878544 | 0.8544315129811997 |
| **BBOW - LR** | **0.8663967611336032** | 0.8639833448993754 | 0.871543577178859 | **0.8677469942498693** |
| **Lexicon - NB** | 0.725752508361204 | 0.7302375044310528 | 0.7210360518025901 | 0.7256076083127861 |
| **Lexicon - LR** | 0.7528604118993135 | 0.7551808921671935 | 0.7525376268813441 | 0.7538569424964937 |
